In [1]:
import numpy as np
import pandas as pd
import cv2
import json
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
## image processing functions to locate game board and label it ##

def findBoard(image, t1, t2):
    """
    Finds location of game baord given input image and crops to the game board.

    Args:
        image: BGR image (as read by cv2.imread)
        threshhold: low threshold value for Canny edge detection
        
    Returns:
        board_region: BGR image cropped to the game board
    """
    
    # convert to gray
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # detect edges
    edges = cv2.Canny(gray, t1, t2)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # crop to game board
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    board_region = image[y:y+h, x:x+w] 

    return board_region


def extractHexVectors(board, xy_to_cubic, color_ranges):
    """
    Extracts color counts for each hex on the game board and outputs them as a vector, using cubic coordinates.

    Args:
        board: BGR image cropped to the board (OpenCV format)
        xy_to_cubic: List of dicts with 'pixel': [x, y], 'cubic': [q, r, s]
        color_ranges: Dict of token_type -> (lower_bgr, upper_bgr)

    Returns:
        List of dicts: [{'coord': [q, r, s], 'vector': {'red': ..., 'blue': ..., ...}}, ...]
    """

    # Convert to RGB
    board = cv2.cvtColor(board, cv2.COLOR_BGR2RGB)

    # Convert list of dicts to a proper mapping for lookup
    xy_to_cubic_dict = {
        tuple(entry["pixel"]): tuple(entry["cubic_coord"])
        for entry in xy_to_cubic
    }

    vectors = []

    for (hex_x, hex_y), cubic_coord in xy_to_cubic_dict.items():
        roi = board[hex_y - 25: hex_y + 25, hex_x - 10: hex_x + 10]  # Region of interest
        vector = {}

        for token_type, (lower, upper) in color_ranges.items():
            lower_np = np.array(lower, dtype=np.uint8)
            upper_np = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(roi, lower_np, upper_np)
            pixel_count = cv2.countNonZero(mask)

            # Special adjustment for spirit token (similar color to board)
            if token_type == "spirit":
                board_mask = cv2.inRange(roi, np.array(color_ranges["board"][0]), np.array(color_ranges["board"][1]))
                pixel_count = max(0, pixel_count - cv2.countNonZero(board_mask))

            vector[token_type] = int(pixel_count)

        vectors.append({
            "coord": list(cubic_coord),  # Output in [q, r, s]
            "vector": vector
        })

    return vectors



def classify_vector(input_vector, token_avgs):
    """
    Given a pixel count vector, return the top matching label from a vector database of average values.
    
    Args:
        input_vector (dict): e.g., {'red': 0, 'blue': 172, ..., 'animal': 133}
        token_vectors (pd.DataFrame) - database of tokens vectors, averaged from labeled data

    Returns:
        list of tuples: [(label, similarity_score), ...]
    """
    
    # ensure vectors are in the same order
    token_order = token_avgs.index.tolist()
    input_array = np.array([[input_vector.get(token, 0) for token in token_order]])  # shape (1, n_tokens)

    # transpose for cosine similarity: now shape (n_labels, n_tokens)
    label_vectors = token_avgs.T.values
    label_names = token_avgs.columns.tolist()

    # cosine similarity
    similarities = cosine_similarity(input_array, label_vectors)[0]
    best_index = similarities.argmax()

    return label_names[best_index]

def convert_to_board_dict(label_list):
    """
    Convert list of {'coord': [...], 'label': 'token1, token2, ...'} 
    to { (q, r, s): [token1, token2, ...], ... }
    """
    board_dict = {}
    for entry in label_list:
        coord = tuple(entry["coord"])
        label = entry["label"]
        if label.lower() == "none":
            board_dict[coord] = []
        else:
            tokens = [token.strip() for token in label.split(",")]
            board_dict[coord] = tokens
    return board_dict


In [3]:
## functions to match the labeled board with spirit and animal patterns ##
def find_spirit_cube(board_dict):
    """
    Returns the cubic coordinate (q, r, s) of the spirit cube.
    """
    for coord, stack in board_dict.items():
        if stack and stack[0] == "spirit":
            return coord
    return None
    
def find_animal_cube(board_dict):
    """
    Returns a list of cubic coordinates (q, r, s) that have an animal cube on top.
    """
    return [coord for coord, stack in board_dict.items() if stack and stack[0] == "animal"]

def rotate_cubic(dq, dr, ds, times=1):
    for _ in range(times % 6):
        dq, dr, ds = -dr, -ds, -dq
    return dq, dr, ds

def rotate_pattern(pattern, times):
    """Rotate spirit/animal pattern"""
    rotated = []
    for entry in pattern:
        dq, dr, ds = entry["dq"], entry["dr"], entry["ds"]
        rotated_entry = dict(entry)  # shallow copy
        rotated_entry["dq"], rotated_entry["dr"], rotated_entry["ds"] = rotate_cubic(dq, dr, ds, times)
        rotated.append(rotated_entry)
    return rotated

def clean_stack(stack):
    """Remove animal/spirit cubes from the top of the stack (if present)."""
    return [item for item in stack if item not in ("animal", "spirit")]

def match_pattern(board_dict, anchor_coord, pattern):
    """
    Check if a rotated pattern matches the board at a given anchor coordinate.

    Args:
        board_dict: dict with keys as (q, r, s) and values as stack lists (bottom to top)
        anchor_coord: (q, r, s) tuple
        pattern: List of dicts with dq, dr, ds and terrain/stack_condition/cube

    Returns:
        True if pattern matches, False otherwise
    """
    for entry in pattern:
        print('Testing pattern entry:', entry)
        dq, dr, ds = entry["dq"], entry["dr"], entry["ds"]
        q, r, s = anchor_coord[0] + dq, anchor_coord[1] + dr, anchor_coord[2] + ds
        coord = (q, r, s)
        print(f'Coordinate tested: {coord}')
        
        stack = board_dict.get(coord, [])
        print(f'Stack before cleaning: {stack}')

        cleaned_stack = clean_stack(stack)
        print(f'Stack after cleaning: {cleaned_stack}')

        # Check cube requirement
        if entry.get("cube", False):
            if not stack or stack[0] not in ["animal", "spirit"]:
                print(f"❌ Cube not found where required at {coord}")
                return False


        # Check simple terrain match (expecting a single-token stack)
        if "terrain" in entry:
            terrain_matches = len(cleaned_stack) == 1 and cleaned_stack[0] == entry["terrain"]
            if not terrain_matches:
                print(f"❌ Terrain mismatch at {coord}. Expected single {entry['terrain']}, got {cleaned_stack}")
                return False


        # Check stack condition match
        if "stack_condition" in entry:
            sc = entry["stack_condition"]
            print('stack_condition:')
            print(sc)
            print('cleaned_stack:')
            print(cleaned_stack)
            if len(cleaned_stack) < sc.get("height", 0):
                print(f"❌ Stack too short at {coord}. Needed height {sc.get('height')}, got {len(cleaned_stack)}")
                return False

            # Check top
            top_matches = cleaned_stack[0] == sc["top"]
            print(f'Top matches: {top_matches}')

            # Optional middle
            middle_matches = True
            if "middle" in sc and len(cleaned_stack) > 1:
                middle_matches = cleaned_stack[1] == sc["middle"]
                print(f'Middle matches or NA: {middle_matches}')

            # Check bottom
            bottom_matches = cleaned_stack[-1] in sc["bottom_options"]
            print(f'Bottom matches: {bottom_matches}')

            if not (top_matches and middle_matches and bottom_matches):
                print(f"❌ Stack structure mismatch at {coord}. Stack: {cleaned_stack}")
                return False

    return True  # All checks passed
def match_pattern_any_rotation(board_dict, pattern):
    """
    Try all 6 rotations of the pattern at every hex in the board_dict.

    Returns:
        Tuple (True, matched_anchor_coord) if a match is found
        Else, (False, None)
    """
    for anchor_coord in board_dict.keys():
        print(f"Testing hex at position: {anchor_coord}")
        for rotation in range(6):
            rotated = rotate_pattern(pattern, rotation)

            print(f"\n🔄 Trying rotation {rotation} at anchor {anchor_coord}")
            print("Rotated Pattern:")
            for p in rotated:
                print(p)
            
            if match_pattern(board_dict, anchor_coord, rotated):
                return True, anchor_coord
    return False, None

def match_patterns_at_cube(board_dict, cube_coord, patterns):
    matches = []
    for pattern in patterns:
        print(f"Testing pattern: {pattern['animal']}")
        for rotation in range(6):
            rotated = rotate_pattern(pattern["pattern"], rotation)
            if match_pattern(board_dict, cube_coord, rotated):
                matches.append({
                    "animal": pattern["animal"],
                    "anchor": cube_coord,
                    "rotation": rotation
                })
    return matches

def resolve_conflicts(candidates, type_name="spirit"):
    """
    Resolve matching conflicts for spirit or animal pattern matches.

    Args:
        candidates (list): List of dicts with keys 'animal', 'anchor', and 'rotation'
        type_name (str): "spirit" or "animal" (used in prompts)

    Returns:
        dict: The selected match {'animal': ..., 'anchor': ..., ...}, or None if no matches
    """
    if not candidates:
        print(f"❌ No {type_name} pattern matched.")
        return None

    # Group by unique animal name
    unique_animals = {}
    for candidate in candidates:
        key = candidate["animal"]
        if key not in unique_animals:
            unique_animals[key] = candidate  # Save first instance

    if len(unique_animals) == 1:
        selected_match = next(iter(unique_animals.values()))
        print(f"✅ {type_name.capitalize()} matched: {selected_match['animal']} at {selected_match['anchor']}")
        return selected_match

    # Conflict: multiple distinct animals matched
    print(f"⚠️ Multiple {type_name} patterns matched:")
    for i, (animal, match) in enumerate(unique_animals.items(), 1):
        print(f"{i}. {animal} at {match['anchor']} (rotation: {match.get('rotation', 'N/A')})")

    selected = int(input(f"Select the correct {type_name} by number: ")) - 1
    selected_animal = list(unique_animals.keys())[selected]
    return unique_animals[selected_animal]


In [4]:
# load image and other necessary data
image = cv2.imread('C:/Users/lacto/Documents/GitHub/HarmoniesRender/training_boards/screenshots/Board4.png')

# get reference coordinates for hexagon centers
f = open('hex_positions_cubic.json')
hex_centers = json.load(f)

# get reference patterns
f = open('spirit_patterns_cubic_cubeanchor.json')
spirit_patterns = json.load(f)

f = open('animal_patterns_cubic_cubeanchor.json')
animal_patterns = json.load(f)

# load token color ranges
f = open('token_color_ranges.json')
color_ranges = json.load(f)

# load token vectors
token_avgs = pd.read_csv('average_token_vectors.csv', index_col=0)

In [9]:
## MAIN ##

# crop image to board state
board = findBoard(image, t1 = 50, t2 = 50)

# count number of pixels in each token's color range for each board hex
hex_vectors = extractHexVectors(board, hex_centers, color_ranges)

# use pixel counts to detect tokens, cubes, and the stack order
labels = []
for hex in hex_vectors: # for each board hex
    input_vector = hex["vector"] # get vector of color counts
    label = classify_vector(input_vector, token_avgs) # use cosine similarity to classify vector
    labels.append({
        "coord": hex["coord"],
        "label": label
    }) 

board_dict = convert_to_board_dict(labels)

# find the spirit animal
spirit_coord = find_spirit_cube(board_dict)
matches = match_patterns_at_cube(board_dict, spirit_coord, spirit_patterns)
matched_spirit = resolve_conflicts(matches, type_name='spirit')

# find placed animals
animal_coords = find_animal_cube(board_dict)

matched_animals = []
for coord in animal_coords:
    matches = match_patterns_at_cube(board_dict, coord, animal_patterns)
    matched_animal = resolve_conflicts(matches, type_name='animal')
    matched_animals.append(matched_animal)

    
## GEN AI ##
    
# infer terrain and habitat features from
# - Spirt Card
# - Animal Cards
# - Tokens (Green, Yellow, Blue, Gray, Red, Brown)
# - Token density

# infer arrangement of features 

# combine inferences with stylistic preferences

# create image

# save image

Testing pattern: domestic cat (spirit)
Testing pattern entry: {'terrain': 'green', 'dq': -1, 'dr': 0, 'ds': 1}
Coordinate tested: (-1, -1, 2)
Stack before cleaning: ['blue']
Stack after cleaning: ['blue']
❌ Terrain mismatch at (-1, -1, 2). Expected single green, got ['blue']
Testing pattern entry: {'terrain': 'green', 'dq': 0, 'dr': -1, 'ds': 1}
Coordinate tested: (0, -2, 2)
Stack before cleaning: ['animal', 'green', 'brown']
Stack after cleaning: ['green', 'brown']
❌ Terrain mismatch at (0, -2, 2). Expected single green, got ['green', 'brown']
Testing pattern entry: {'terrain': 'green', 'dq': 1, 'dr': -1, 'ds': 0}
Coordinate tested: (1, -2, 1)
Stack before cleaning: ['animal', 'gray']
Stack after cleaning: ['gray']
❌ Terrain mismatch at (1, -2, 1). Expected single green, got ['gray']
Testing pattern entry: {'terrain': 'green', 'dq': 1, 'dr': 0, 'ds': -1}
Coordinate tested: (1, -1, 0)
Stack before cleaning: ['animal', 'red', 'gray']
Stack after cleaning: ['red', 'gray']
❌ Terrain misma

Select the correct animal by number:  2


Testing pattern: ray
Testing pattern entry: {'dq': 0, 'dr': 0, 'ds': 0, 'terrain': 'blue', 'cube': True}
Coordinate tested: (1, -2, 1)
Stack before cleaning: ['animal', 'gray']
Stack after cleaning: ['gray']
❌ Terrain mismatch at (1, -2, 1). Expected single blue, got ['gray']
Testing pattern entry: {'dq': 0, 'dr': 0, 'ds': 0, 'terrain': 'blue', 'cube': True}
Coordinate tested: (1, -2, 1)
Stack before cleaning: ['animal', 'gray']
Stack after cleaning: ['gray']
❌ Terrain mismatch at (1, -2, 1). Expected single blue, got ['gray']
Testing pattern entry: {'dq': 0, 'dr': 0, 'ds': 0, 'terrain': 'blue', 'cube': True}
Coordinate tested: (1, -2, 1)
Stack before cleaning: ['animal', 'gray']
Stack after cleaning: ['gray']
❌ Terrain mismatch at (1, -2, 1). Expected single blue, got ['gray']
Testing pattern entry: {'dq': 0, 'dr': 0, 'ds': 0, 'terrain': 'blue', 'cube': True}
Coordinate tested: (1, -2, 1)
Stack before cleaning: ['animal', 'gray']
Stack after cleaning: ['gray']
❌ Terrain mismatch at (1

In [ ]:

# TODO: put pattern matching in a loop and have it output all possibilities, then prompt user to resolve conflicts.
# this will prepare for animal matching, where there will be many more conflicts

# TODO: need to make sure pattern isn't matched on same cube after the fact

In [8]:
cv2.imshow('', board)
cv2.waitKey(0)

-1

In [13]:
for match in matched_animals:
    animal = match['animal']
    anchor = match['anchor']
    print(f"{animal.capitalize()} at {anchor}")

Macaw at (-2, -1, 3)
Penguin at (-2, 1, 1)
Penguin at (-1, 0, 1)
Macaque at (-1, 2, -1)
Macaw at (0, -2, 2)
Macaw at (0, 0, 0)
Penguin at (1, -2, 1)
Gecko at (1, -1, 0)
Ladybug at (1, 0, -1)
Ladybug at (1, 1, -2)


In [15]:
print(spirit_coord)

(0, -1, 1)
